<a href="https://colab.research.google.com/github/GH-6021/face-emotion-recognition/blob/main/src/mobileNet_fineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import google.protobuf

print("TensorFlow version:", tf.__version__)
print("protobuf version:", google.protobuf.__version__)


TensorFlow version: 2.18.0
protobuf version: 5.29.3


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import os

# 삭제할 폴더들이 있는 상위 디렉터리 (Colab 환경에 맞춰서 변경)
base_dir = "/content/train/Training"

# 감정 폴더 리스트
emotion_folders = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sadness", "Surprise"]

# 각 폴더를 순회하며 파일 개수를 조정
for emotion in emotion_folders:
    folder_path = os.path.join(base_dir, emotion)

    if not os.path.exists(folder_path):
        print(f"폴더 없음: {folder_path}")
        continue

    # 폴더 내 모든 파일 목록 가져오기 (경로 포함)
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # 파일이 5000개 이하이면 삭제할 필요 없음
    if len(files) <= 10000:
        print(f"{emotion} 폴더는 이미 5000개 이하입니다. ({len(files)}개)")
        continue

    # 파일을 생성 시간 순으로 정렬 (오래된 파일 먼저 삭제)
    files.sort(key=os.path.getctime)

    # 초과하는 파일 개수 계산
    excess_files = len(files) - 10000

    # 초과된 파일 삭제
    for file in files[:excess_files]:
        os.remove(file)

    print(f"{emotion} 폴더에서 {excess_files}개의 파일을 삭제 완료! 현재 남은 파일: 5000개")


Angry 폴더는 이미 5000개 이하입니다. (10000개)
Disgust 폴더는 이미 5000개 이하입니다. (10000개)
Fear 폴더는 이미 5000개 이하입니다. (10000개)
Happiness 폴더는 이미 5000개 이하입니다. (10000개)
Neutral 폴더는 이미 5000개 이하입니다. (10000개)
Sadness 폴더는 이미 5000개 이하입니다. (10000개)
Surprise 폴더에서 1311개의 파일을 삭제 완료! 현재 남은 파일: 5000개


In [60]:
!unzip /content/drive/MyDrive/train.zip

Archive:  /content/drive/MyDrive/train.zip
replace train/Training/Angry/d01097b8d1b0ed0f08ad6fc81ad2d1f6b562df02a21b017264c28eb59f7f342d_여_20_분노_공공시설&종교&의료시설_20201207150447-001-001.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [71]:
!unzip -j /content/drive/MyDrive/train.zip "train/Training/Surprise/*" -d /content/train/Training/Surprise


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144756-002-017.jpg  
  inflating: /content/train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144756-002-018.jpg  
  inflating: /content/train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-001.jpg  
  inflating: /content/train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-002.jpg  
  inflating: /content/train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-003.jpg  
  inflating: /content/train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-004.jpg  
  inflating: /content/train/Training/Sur

In [66]:
import shutil

# 삭제할 폴더 경로
folder_path = '/content/train/Training/Happiness'

# 폴더 및 그 안의 모든 파일 삭제
shutil.rmtree(folder_path)


In [ ]:
!unzip /content/drive/MyDrive/validation.zip

In [ ]:
from tensorflow.keras.models import load_model

# 기존 모델 로드
model = load_model(r'C:\baekjoon_Git_repository\face-emotion-recognition\models\affectnet_emotions\mobilenet_7.h5')

# 모델 구조 확인
model.summary()


Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)            │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_pad (ZeroPadding2D)       │ (None, 225, 225, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,493,063 (13.32 MB)

 Trainable params: 3,471,175 (13.24 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

conv_output = model.get_layer('conv_pw_13_relu').output
x = GlobalAveragePooling2D()(conv_output)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
num_classes = 7
predictions = Dense(num_classes, activation='softmax')(x)

# 새로운 모델 정의
fine_tuned_model = Model(inputs=model.input, outputs=predictions)

# 일부 레이어만 동결 해제 (마지막 50개 레이어 학습 가능)
for layer in model.layers[:-50]:  # 처음 ~ (끝에서 50개 이전) 레이어는 동결
    layer.trainable = False
for layer in model.layers[-50:]:  # 마지막 50개 레이어는 학습 가능
    layer.trainable = True

# 모델 컴파일 (learning_rate 작게 설정)
fine_tuned_model.compile(optimizer=Adam(learning_rate=1e-5),  # 작은 학습률로 미세 조정
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

fine_tuned_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)            │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_pad (ZeroPadding2D)       │ (None, 225, 225, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,360,967 (12.82 MB)

 Trainable params: 132,103 (516.03 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

/////////

In [ ]:
# from PIL import Image, ExifTags
# import os

# def clean_exif_and_save(image_path):
#     """ 손상된 EXIF 데이터를 제거하고 이미지 저장 """
#     try:
#         with Image.open(image_path) as img:
#             img_without_exif = img.copy()
#             img_without_exif.info.pop("exif", None)  # EXIF 제거
#             img_without_exif.save(image_path)  # 다시 저장
#     except Exception as e:
#         print(f"이미지 처리 중 오류 발생: {image_path} -> {e}")

# def process_directory(directory):
#     """ 디렉토리 내 모든 이미지의 EXIF 데이터를 정리 """
#     for subdir, _, files in os.walk(directory):
#         for file in files:
#             file_path = os.path.join(subdir, file)
#             clean_exif_and_save(file_path)

# process_directory("D:/train/Training")
# process_directory("C:/baekjoon_Git_repository/face-emotion-recognition/src/mobileNet_fineTuning_dataset/validation")


In [ ]:
# import os
# from PIL import Image

# def remove_corrupt_images(directory):
#     for subdir, _, files in os.walk(directory):
#         for filename in files:
#             file_path = os.path.join(subdir, filename)
#             try:
#                 with Image.open(file_path) as img:
#                     img.verify()  # 첫 번째 검증 (메타데이터 확인)
#                 with Image.open(file_path) as img:
#                     img.convert("RGB")  # 두 번째 검증 (실제 픽셀 데이터 확인)
#             except (OSError, IOError):
#                 print(f"손상된 이미지 삭제: {file_path}")
#                 os.remove(file_path)

# train_dir = 'D:/train/Training'
# valid_dir = 'C:/baekjoon_Git_repository/face-emotion-recognition/src/mobileNet_fineTuning_dataset/validation'

# remove_corrupt_images(train_dir)
# remove_corrupt_images(valid_dir)


/////////

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.8  # 학습 데이터의 10%만 사용
)

val_datagen = ImageDataGenerator(rescale=1./255)

valid_dir = 'D:/train/Training'
train_dir = 'D:/validation'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical',
    shuffle=True,
    subset='training'  # 학습 데이터를 일부만 사용
)

val_generator = val_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# 모델 학습
history = fine_tuned_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Found 16631 images belonging to 7 classes.
Found 44625 images belonging to 7 classes.
Epoch 1/10
 346/2079 ━━━━━━━━━━━━━━━━━━━━ 23:36 818ms/step - accuracy: 0.1588 - loss: 1.9473

KeyboardInterrupt: 

In [ ]:
#모델저장
fine_tuned_model.save('fine_tuned_model.h5')

In [ ]:
#모델평가
loss, accuracy = fine_tuned_model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")